<a href="https://colab.research.google.com/github/gatherheart/Classification_With_BERT/blob/main/Khaii_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import glob
import pandas as pd
import matplotlib.pyplot as plt

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3
!pip install torch

     |████████████████████████████████| 55.0MB 104kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 348kB 13.2MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp36-cp36m-linux_x86_64.whl size=588515 sha256=3af5dd0e1d44724bfb662efc20cc20e1ca8f7fe437b531fca8824455612e617f
  Stored in directory: /root/.cache/pip/wheels/37/65/52/63032864a0f31a08b9a88569f803b5bafac8abd207fd7f7534
Successfully built gluonnlp
     |████████████████████████████████| 1.1MB 12.7MB/s 
     |████████████████████████████████| 757kB 13.2MB/s 
     |████████████████████████████████| 3.0MB 31.1MB/s 
     |████████████████████████████████| 890kB 37.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=ff504b89c0244bfd89d44e863f3ffea40b96ee3d457d230f35673c50a190b705
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a

In [9]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-818nry6s
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-818nry6s
  Created wheel for kobert: filename=kobert-0.1.2-cp36-none-any.whl size=12734 sha256=517e5900f82e2355a83cefd9d543c4f7fff4144aeb5e61ffb6bdaf2939ca657c
  Stored in directory: /tmp/pip-ephem-wheel-cache-4zk_86gj/wheels/a2/b0/41/435ee4e918f91918be41529283c5ff86cd010f02e7525aecf3
Successfully built kobert


In [10]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
from tqdm import tqdm, tqdm_notebook, notebook

In [11]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [12]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences

In [13]:
BASE_PATH = '/content/drive/MyDrive/hyun_data'

In [14]:
data_2019 = glob.glob(BASE_PATH+'/crawling/NewsResult_2019*')

In [15]:
data_2019

['/content/drive/MyDrive/hyun_data/crawling/NewsResult_20190801-20190810.csv',
 '/content/drive/MyDrive/hyun_data/crawling/NewsResult_20190721-20190731.csv',
 '/content/drive/MyDrive/hyun_data/crawling/NewsResult_20190711-20190720.csv',
 '/content/drive/MyDrive/hyun_data/crawling/NewsResult_20190811-20190820.csv',
 '/content/drive/MyDrive/hyun_data/crawling/NewsResult_20190821-20190831.csv',
 '/content/drive/MyDrive/hyun_data/crawling/NewsResult_20190901-20190910.csv',
 '/content/drive/MyDrive/hyun_data/crawling/NewsResult_20190921-20190930.csv',
 '/content/drive/MyDrive/hyun_data/crawling/NewsResult_20190911-20190920.csv',
 '/content/drive/MyDrive/hyun_data/crawling/NewsResult_20190701-20190710.csv']

In [16]:
data_2020 = glob.glob(BASE_PATH+'/crawling/NewsResult_2020*')

In [17]:
data_2020

['/content/drive/MyDrive/hyun_data/crawling/NewsResult_20200711-20200720.csv',
 '/content/drive/MyDrive/hyun_data/crawling/NewsResult_20200701-20200710.csv',
 '/content/drive/MyDrive/hyun_data/crawling/NewsResult_20200721-20200731.csv',
 '/content/drive/MyDrive/hyun_data/crawling/NewsResult_20200801-20200810.csv',
 '/content/drive/MyDrive/hyun_data/crawling/NewsResult_20200811-20200820.csv',
 '/content/drive/MyDrive/hyun_data/crawling/NewsResult_20200821-20200831.csv',
 '/content/drive/MyDrive/hyun_data/crawling/NewsResult_20200901-20200910.csv',
 '/content/drive/MyDrive/hyun_data/crawling/NewsResult_20200911-20200920.csv',
 '/content/drive/MyDrive/hyun_data/crawling/NewsResult_20200921-20200930.csv']

In [18]:
df = pd.concat(map(lambda x: pd.read_csv(x), data_2019))

In [19]:
df

,0,뉴스 식별자,일자,단어,본문
0,1,2.100601e+06,20190810,"수,다,사진,다나,인스타그램,가수,모습,공개,다,일,오후,자신,사회,관계,망,서비스...","['\n', '\n', '\n', '\n', '\n', '가수 다나/사진=다나 인스..."
1,2,2.100601e+06,20190810,"왓,사진,방송,영상,캡처,진,범,비리장부,서강준,습격,일,방송,주말드라,왓,김영군,...","['\n', '\n', '\n', '\n', '\n', '\n', '왓쳐 (사진=방..."
2,3,2.100601e+06,20190810,"호텔,델루,사진,방송,영상,캡처,호텔,델루,남,우물,신,특,연,일,방송,토일,드라마...","['\n', '\n', '\n', '\n', '\n', '\n', '호텔 델루나 (..."
3,4,2.100601e+06,20190810,"오승환,사진,연합,뉴스,돌부,처,오승환,정팀,삼성,라이온즈,유니폼,오승환,일,대구,...","['\n', '\n', '\n', '\n', '\n', '오승환/사진=연합뉴스', ..."
4,5,2.100601e+06,20190810,"호텔,델루,사진,방송,영상,캡처,여,진,구,개장,호텔,델루나,이지,일,오후,방송,호...","['\n', '\n', '\n', '\n', '\n', '\n', '호텔 델루나 (..."
...,...,...,...,...,...
914,915,2.100601e+06,20190709,"사진,연합,뉴스,일본,정부,한국,경제,보복,조치,대북제재,위반,의혹,들,청와대,강경...","['\n', '\n', '\n', '\n', '\n', '사진=연합뉴스', '\n'..."
915,916,2.100601e+06,20190709,"이낙연,국무총리,한국,수출,규제,북한,연결,일본,아베,총리,발언,위험,발언,말,총리...","['\n', '\n', '\n', '\n', '\n', '\n', '\n ..."
916,917,2.100601e+06,20190709,"양호석,차오름,사진,양호석,차오름,인스타그램,양호석,차오름,진흙탕,폭로전,일,서울중...","['\n', '\n', '\n', '\n', '\n', '양호석, 차오름/사진=양호..."
917,918,2.100601e+06,20190709,"경기,용인시,시청,개,구청,개,읍,면,동,동시,실시간,연결,영상회,시스템,구축,시정...","['\n', '경기 용인시는 시청과 ', '3', '개 구청', ', 31', '개..."


In [17]:
df['단어'] =  list(map(lambda x: str(x).replace(',', ' '), df['단어']))

In [18]:
df = df.reset_index().drop(['0'], axis=1)

In [19]:
df.head(5)

,index,뉴스 식별자,일자,단어,본문
0,0,2.100601e+06,20190810,수 다 사진 다나 인스타그램 가수 모습 공개 다 일 오후 자신 사회 관계 망 서비스...,"['\n', '\n', '\n', '\n', '\n', '가수 다나/사진=다나 인스..."
1,1,2.100601e+06,20190810,왓 사진 방송 영상 캡처 진 범 비리장부 서강준 습격 일 방송 주말드라 왓 김영군 ...,"['\n', '\n', '\n', '\n', '\n', '\n', '왓쳐 (사진=방..."
2,2,2.100601e+06,20190810,호텔 델루 사진 방송 영상 캡처 호텔 델루 남 우물 신 특 연 일 방송 토일 드라마...,"['\n', '\n', '\n', '\n', '\n', '\n', '호텔 델루나 (..."
3,3,2.100601e+06,20190810,오승환 사진 연합 뉴스 돌부 처 오승환 정팀 삼성 라이온즈 유니폼 오승환 일 대구 ...,"['\n', '\n', '\n', '\n', '\n', '오승환/사진=연합뉴스', ..."
4,4,2.100601e+06,20190810,호텔 델루 사진 방송 영상 캡처 여 진 구 개장 호텔 델루나 이지 일 오후 방송 호...,"['\n', '\n', '\n', '\n', '\n', '\n', '호텔 델루나 (..."


In [20]:
df.iloc[9]['본문']

'[\'\\n\', \'\\n\', \'\\n\', \'\\n\', \'\\n\', \'일본 DHC/사진=JTBC\', \'\\n\', \'\\n                        일본 유명 화장품업체인 DHC의 자회사인 방송국이 혐한 방송을 진행, 논란이 일고 있다.\', "JTBC는 10일 DHC의 자회사 \'DHC텔레비선\'의 한 프로그램에서 한국의 불매운동을 비하하고 역사왜곡 발언을 했다고 보도했다.", \'보도에 따르면 이 방송에 출연한 한 패널은 "한국은 원래 금방 뜨거워지고 금방 식는 나라니까. 일본은 그냥 조용히 두고 봐야지"라며 한국의 일본상품 불매운동을 비하했다.\', \'또 다른 패널은 "\\\'조센징\\\'들은 한문을 썼는데 한문을 문자화시키지 못해서, 일본에서 만든 교과서로 한글을 배포했다. 일본인이 한글을 통일시켜서 지금의 한글이 됐다"고 역사왜곡 발언을 했다.\', "더불어 인터넷을 기반으로 하는 \'DHC텔레비전\'은 단순한 자회사가 아니라 중간 중간 자사 화장품 광고도 내보내는 등 해당 브랜드와 밀접한 관련을 맺고 있다.", \'나아가 요시다 요시아키 DHC 회장은 과거 재일동포를 비하하거나 극우 정당을 지원했다는 논란 등으로 비판을 받기도 했다.\', \'김정호 한경닷컴 기자 newsinfo@hankyung.com\\n\\t\\t\\t\\t\\t\\t\\t\\t\']'

In [28]:
NUM_LABELS = 58

In [29]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=NUM_LABELS,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
bertmodel, vocab = get_pytorch_kobert_model()

[███████████████████████████████████████...........]

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

In [ ]:
model.cuda()
model.load_state_dict(torch.load(PATH))
model.eval()

In [31]:
!pip install textrankr

In [41]:
your_text_here = '세계 각국의 ‘고부가가치’를 향한 경쟁이 치열하다. 무역 자체도 이미 부가가치 무역으로 탈바꿈한 상태다. 그만큼 공급망이 복잡해지고 기술과 지식이 중요해지는 세상이다. 그 속에서 한국 무역은 ‘세계 경제의 바로미터’로 인정받고 있다. 글로벌 공급망에 완전히 노출돼 있고, 이미 높은 부가가치 제품을 수출하는 국가에 속한다. 하지만 좀 더 큰 부가가치를 올리기 위해 다른 나라들과 치열하게 경쟁하고 있다. 4차 산업혁명과 바이오 등 무궁무진한 분야의 주도권을 쥐기 위한 레이스다. 이 와중에 일본의 반도체 수출규제란 걸림돌이 생겼다. 한국 부가가치 수출의 현주소와 전망 등을 짚어본다.', '지난달 발간된 일본 경제산업성의 통상백서는 아세안(동남아국가연합) 국가들의 전자기기와 부품 수요를 한국에 빼앗길 가능성에 대해 언급했다. 일본의 장래 시장으로 여겼던 아세안에서 한국 등에 선수를 빼앗길까 노심초사하는 모습이 역력하다. 그 이면에는 글로벌 가치사슬에 따른 고부가가치 품목의 수출이 자리 잡고 있다. 백서는 한국 제조업 생산에서 차지하는 수출품의 부가가치 비중이 2005년 19.2%, 2015년엔 22.4%나 된다고 지적했다. 일본의 16.1%(2015년)보다 훨씬 높다는 점을 내비치고 있다.', '갈수록 무역에서 부가가치가 중요해지고 있다. 애플이 제조하는 스마트폰인 아이폰에서 중국의 노동력이 차지하는 비중은 1.8%에 불과하다는 통계도 있다. 한국 기업을 포함한 수백 개 기업이 아이폰 제조에 참여하면서 부가가치를 창출해낸다. 물론 높은 부가가치는 지식집약과 자본집약의 산물이다. 자동차도 그렇고 반도체도 그렇다. 높은 부가가치를 창출하는 품목과 산업의 주도권을 누가 쥐느냐에 세계 무역의 향방이 결정된다. 세계 기업들은 그런 방향에 맞춰 얽히고설켜 있다. 글로벌 가치사슬(GVC) 생태계다. 경제협력개발기구(OECD)는 각국의 무역 금액 외에 부가가치무역(TiVA) 통계를 따로 내고 있다.\n ', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '반도체·정밀기계 부가가치화 주도', '이런 부가가치 무역에서 한국의 위상은 높아지고 있다. 당장 세계 무역에서 가장 중요한 중국에서 그렇다. OECD 자료에 따르면 중국이 수출하는 제품에 들어가는 중간재의 연간 부가가치는 한국이 426억달러(2.2%)로 1위다. 일본의 346억달러에 비해 80억달러 많다. 반도체나 정밀기계 화학제품 등 각종 부품과 소재가 중국으로 수출되고 있다. 중국은 이들을 받아 완제품으로 만들어 미국과 유럽에 수출한다. 이런 제품들은 자본집약적이고 지식집약적이다.', '불과 20년 전만 하더라도 고부가가치 무역은 어려운 과제였다. 오히려 일본에서 첨단소재와 부품을 들여와 수출 제품을 만들었다. 일본 닛세이기초연구소 자료에 따르면 한국의 전자부품과 통신기기에서 차지하는 일본 부품의 부가가치는 1995년 기준 20%를 넘는 게 수두룩하다. 지금은 모든 품목의 일본 부가가치 평균이 5%에도 못 미친다. 자동차 부품은 이미 일본과 경합구도를 만들어내고 있으며 전자 부품 또한 많이 줄어들고 있다. 그만큼 부품의 국산화가 이뤄졌거나 다른 국가 제품으로 방향을 틀었다.', '물론 한국 부가가치 품목 수출에서 중국의 성장을 빼놓을 수 없다. ‘세계의 공장’ 중국은 풍부한 노동력을 바탕으로 세계의 제조 기업들을 불러들였다. 제품을 만들기 위한 각종 기자재, 부품, 원료 등을 가까운 한국에서 찾았다. 가격경쟁력이나 품질경쟁력이 있었기 때문에 가능한 일이었다. 디지털도 한몫했다. 한국에서 디지털 기기의 급속한 보급은 노동력 절감과 생산성 향상으로 이어졌다. 원가 경쟁력이 충분히 생겼다. 도표에 따르면 부가가치를 높이는 산업용 로봇 활용 비중에서 한국이 월등하다. 독일조차 한국의 절반이다.', '고부가가치화를 향한 선택과 집중 전략을 택하고 자급보다 글로벌화에 기울였던 기업 전략이 먹혀든 것이다. 국내 기업들은 전 세계 시장 개척에 나섰고 세계 1등 제품을 모색했다. 그렇게 해서 반도체 디스플레이 등 세계 1등 부품과 제품이 탄생했다.', '외국에선 중국의 사드(고고도 미사일방어체계) 보복 사태에서 한국 기업들의 발 빠른 움직임을 눈여겨본다. 사드 사태가 터진 이후 한국 기업들은 기민하게 중국에 치우쳤던 생산 거점을 분산시켰다. 그만큼 한국 기업의 위기 대응능력이 뛰어나다는 것이다.', '지금 세계 무역의 퇴조가 확연하다. 네덜란드의 세계무역국 자료에 따르면 지난 5월 세계 무역량은 전년 동기 대비 0.4% 줄어들었다. 독일과 일본도 감소했으며 한국 또한 줄었다. 한국의 7월 수출은 전년 동기 대비 11% 감소했다. 한국의 부가가치 품목 수출도 떨어지고 있다.', '자동차 부품은 일본과 경합', '한국에서 반도체 수출이 감소하면 주변국도 타격을 받는다. 일본의 반도체 제조장치와 주변부품 수요가 줄어든다. 시진핑 중국 국가주석은 이런 때 자력갱생을 외치고 중국 내 자급경제를 독촉하고 있다. 자칫 한국의 대중 수출에 큰 영향을 받을 수 있다. 하지만 이런 위기일수록 기회가 마련된다는 주장도 있다. 일본의 한 전문가는 중국 시장에 대한 미국 기업의 경쟁력 저하가 한국 기업에 이득이 될 것이라고 본다.', '산업통상자원부도 최근 내놓은 보고서에서 “미·중 무역분쟁 속에 중국의 부품·소재산업 자급률 향상이 한국에 위기이자 기회가 되고 있다”고 전제한다. 보고서는 이어 “중국은 한국의 대(對)중국 주력 수출제품인 철강, 석유화학에서 이미 자급 생산체제를 갖췄다고 평가되며, 이제는 반도체의 자급 준비도 착착 진행하고 있다”며 “그동안 중국에 중간재를 공급하던 한국, 일본, 대만과 중국 간의 분업 협력구조가 깨지면서 무한 경쟁에 직면할 것으로 보인다”고 내다봤다.', '보고서는 중국의 부품 자급률이 높아질 때일수록 한국이 소재와 장비를 공급하는 망을 형성해야 한다고 주장한다. 우리의 조립·부품산업이 일본의 소재·장비를 공급받아 성장했듯이 우리도 중국에 대해 일본과 같은 역할을 할 수 있어야 한다는 것이다.', '한·일·중·대만 지역협력 깨질 듯', '한·일 마찰도 마찬가지다. 일본 정부가 반도체 재료를 수출 규제한다면 가치사슬 체계는 달라지고 부가가치 시스템도 흔들린다. 이럴 때일수록 새로운 공급망을 만들거나 보다 나은 소재를 개발할 수도 있다. 이들 모두 기업이 할 수 있고 해야 할 몫이다.', '더욱 중요한 문제는 미래를 향한 부가가치를 누가 확보하느냐에 달려 있다. 산업부 보고서에도 “전기자동차, 스마트와 같은 신산업은 아직 권역별 가치사슬이 형성돼 있지 않다”며 “새롭게 형성되는 가치사슬을 우리가 주도할 수 있는 기회를 잡아야 한다”고 밝히고 있다.', '최근 일본 정부와 재계 언론은 하나같이 ‘디지털’을 외치고 있다. 디지털 지체로 생긴 한국과의 부가가치 열위를 만회하겠다고 읽힌다. 새로운 가치사슬과 부가가치를 향해 애쓰는 노력이 엿보인다. 일본의 수출규제로 촉발된 부품 소재 국산화도 나름 의미 있고 필요한 일이다. 하지만 일본보다 부가가치를 높일 수 있는 5G 등 지식집약적인 4차 산업에 매진하는 것이 더 필요할지 모른다.'

In [36]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 30.6MB/s 
     |████████████████████████████████| 92kB 10.2MB/s 
     |████████████████████████████████| 460kB 42.3MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0


In [37]:
from typing import List
from konlpy.tag import Okt

class OktTokenizer:
    okt: Okt = Okt()

    def __call__(self, text: str) -> List[str]:
        tokens: List[str] = self.okt.phrases(text)
        return tokens

In [48]:
from typing import List
from textrankr import TextRank

mytokenizer: OktTokenizer = OktTokenizer()
textrank: TextRank = TextRank(mytokenizer)

k: int = 3  # num sentences in the resulting summary

summarized: str = textrank.summarize(str(your_text_here), k)
print(summarized)

# if verbose=False, it returns a list
summaries: List[str] = textrank.summarize(str(your_text_here), k, verbose=False)
for summary in summaries:
    print(summary)

', '물론 한국 부가가치 품목 수출에서 중국의 성장을 빼놓을 수 없다
한국의 부가가치 품목 수출도 떨어지고 있다
', '자동차 부품은 일본과 경합', '한국에서 반도체 수출이 감소하면 주변국도 타격을 받는다
', '물론 한국 부가가치 품목 수출에서 중국의 성장을 빼놓을 수 없다
한국의 부가가치 품목 수출도 떨어지고 있다
', '자동차 부품은 일본과 경합', '한국에서 반도체 수출이 감소하면 주변국도 타격을 받는다


In [32]:
pd.read_csv('/content/NewsResult_20190811-20190820 (1).csv')

,0,2100601.2019082,20190820,"['\n', '\n', '\n', '\n', '\n', '\n', '\n 세계 각국의 ‘고부가가치’를 향한 경쟁이 치열하다. 무역 자체도 이미 부가가치 무역으로 탈바꿈한 상태다. 그만큼 공급망이 복잡해지고 기술과 지식이 중요해지는 세상이다. 그 속에서 한국 무역은 ‘세계 경제의 바로미터’로 인정받고 있다. 글로벌 공급망에 완전히 노출돼 있고, 이미 높은 부가가치 제품을 수출하는 국가에 속한다. 하지만 좀 더 큰 부가가치를 올리기 위해 다른 나라들과 치열하게 경쟁하고 있다. 4차 산업혁명과 바이오 등 무궁무진한 분야의 주도권을 쥐기 위한 레이스다. 이 와중에 일본의 반도체 수출규제란 걸림돌이 생겼다. 한국 부가가치 수출의 현주소와 전망 등을 짚어본다.', '지난달 발간된 일본 경제산업성의 통상백서는 아세안(동남아국가연합) 국가들의 전자기기와 부품 수요를 한국에 빼앗길 가능성에 대해 언급했다. 일본의 장래 시장으로 여겼던 아세안에서 한국 등에 선수를 빼앗길까 노심초사하는 모습이 역력하다. 그 이면에는 글로벌 가치사슬에 따른 고부가가치 품목의 수출이 자리 잡고 있다. 백서는 한국 제조업 생산에서 차지하는 수출품의 부가가치 비중이 2005년 19.2%, 2015년엔 22.4%나 된다고 지적했다. 일본의 16.1%(2015년)보다 훨씬 높다는 점을 내비치고 있다.', '갈수록 무역에서 부가가치가 중요해지고 있다. 애플이 제조하는 스마트폰인 아이폰에서 중국의 노동력이 차지하는 비중은 1.8%에 불과하다는 통계도 있다. 한국 기업을 포함한 수백 개 기업이 아이폰 제조에 참여하면서 부가가치를 창출해낸다. 물론 높은 부가가치는 지식집약과 자본집약의 산물이다. 자동차도 그렇고 반도체도 그렇다. 높은 부가가치를 창출하는 품목과 산업의 주도권을 누가 쥐느냐에 세계 무역의 향방이 결정된다. 세계 기업들은 그런 방향에 맞춰 얽히고설켜 있다. 글로벌 가치사슬(GVC) 생태계다. 경제협력개발기구(OECD)는 각국의 무역 금액 외에 부가가치무역(TiVA) 통계를 따로 내고 있다.\n ', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '반도체·정밀기계 부가가치화 주도', '이런 부가가치 무역에서 한국의 위상은 높아지고 있다. 당장 세계 무역에서 가장 중요한 중국에서 그렇다. OECD 자료에 따르면 중국이 수출하는 제품에 들어가는 중간재의 연간 부가가치는 한국이 426억달러(2.2%)로 1위다. 일본의 346억달러에 비해 80억달러 많다. 반도체나 정밀기계 화학제품 등 각종 부품과 소재가 중국으로 수출되고 있다. 중국은 이들을 받아 완제품으로 만들어 미국과 유럽에 수출한다. 이런 제품들은 자본집약적이고 지식집약적이다.', '불과 20년 전만 하더라도 고부가가치 무역은 어려운 과제였다. 오히려 일본에서 첨단소재와 부품을 들여와 수출 제품을 만들었다. 일본 닛세이기초연구소 자료에 따르면 한국의 전자부품과 통신기기에서 차지하는 일본 부품의 부가가치는 1995년 기준 20%를 넘는 게 수두룩하다. 지금은 모든 품목의 일본 부가가치 평균이 5%에도 못 미친다. 자동차 부품은 이미 일본과 경합구도를 만들어내고 있으며 전자 부품 또한 많이 줄어들고 있다. 그만큼 부품의 국산화가 이뤄졌거나 다른 국가 제품으로 방향을 틀었다.', '물론 한국 부가가치 품목 수출에서 중국의 성장을 빼놓을 수 없다. ‘세계의 공장’ 중국은 풍부한 노동력을 바탕으로 세계의 제조 기업들을 불러들였다. 제품을 만들기 위한 각종 기자재, 부품, 원료 등을 가까운 한국에서 찾았다. 가격경쟁력이나 품질경쟁력이 있었기 때문에 가능한 일이었다. 디지털도 한몫했다. 한국에서 디지털 기기의 급속한 보급은 노동력 절감과 생산성 향상으로 이어졌다. 원가 경쟁력이 충분히 생겼다. 도표에 따르면 부가가치를 높이는 산업용 로봇 활용 비중에서 한국이 월등하다. 독일조차 한국의 절반이다.', '고부가가치화를 향한 선택과 집중 전략을 택하고 자급보다 글로벌화에 기울였던 기업 전략이 먹혀든 것이다. 국내 기업들은 전 세계 시장 개척에 나섰고 세계 1등 제품을 모색했다. 그렇게 해서 반도체 디스플레이 등 세계 1등 부품과 제품이 탄생했다.', '외국에선 중국의 사드(고고도 미사일방어체계) 보복 사태에서 한국 기업들의 발 빠른 움직임을 눈여겨본다. 사드 사태가 터진 이후 한국 기업들은 기민하게 중국에 치우쳤던 생산 거점을 분산시켰다. 그만큼 한국 기업의 위기 대응능력이 뛰어나다는 것이다.', '지금 세계 무역의 퇴조가 확연하다. 네덜란드의 세계무역국 자료에 따르면 지난 5월 세계 무역량은 전년 동기 대비 0.4% 줄어들었다. 독일과 일본도 감소했으며 한국 또한 줄었다. 한국의 7월 수출은 전년 동기 대비 11% 감소했다. 한국의 부가가치 품목 수출도 떨어지고 있다.', '자동차 부품은 일본과 경합', '한국에서 반도체 수출이 감소하면 주변국도 타격을 받는다. 일본의 반도체 제조장치와 주변부품 수요가 줄어든다. 시진핑 중국 국가주석은 이런 때 자력갱생을 외치고 중국 내 자급경제를 독촉하고 있다. 자칫 한국의 대중 수출에 큰 영향을 받을 수 있다. 하지만 이런 위기일수록 기회가 마련된다는 주장도 있다. 일본의 한 전문가는 중국 시장에 대한 미국 기업의 경쟁력 저하가 한국 기업에 이득이 될 것이라고 본다.', '산업통상자원부도 최근 내놓은 보고서에서 “미·중 무역분쟁 속에 중국의 부품·소재산업 자급률 향상이 한국에 위기이자 기회가 되고 있다”고 전제한다. 보고서는 이어 “중국은 한국의 대(對)중국 주력 수출제품인 철강, 석유화학에서 이미 자급 생산체제를 갖췄다고 평가되며, 이제는 반도체의 자급 준비도 착착 진행하고 있다”며 “그동안 중국에 중간재를 공급하던 한국, 일본, 대만과 중국 간의 분업 협력구조가 깨지면서 무한 경쟁에 직면할 것으로 보인다”고 내다봤다.', '보고서는 중국의 부품 자급률이 높아질 때일수록 한국이 소재와 장비를 공급하는 망을 형성해야 한다고 주장한다. 우리의 조립·부품산업이 일본의 소재·장비를 공급받아 성장했듯이 우리도 중국에 대해 일본과 같은 역할을 할 수 있어야 한다는 것이다.', '한·일·중·대만 지역협력 깨질 듯', '한·일 마찰도 마찬가지다. 일본 정부가 반도체 재료를 수출 규제한다면 가치사슬 체계는 달라지고 부가가치 시스템도 흔들린다. 이럴 때일수록 새로운 공급망을 만들거나 보다 나은 소재를 개발할 수도 있다. 이들 모두 기업이 할 수 있고 해야 할 몫이다.', '더욱 중요한 문제는 미래를 향한 부가가치를 누가 확보하느냐에 달려 있다. 산업부 보고서에도 “전기자동차, 스마트와 같은 신산업은 아직 권역별 가치사슬이 형성돼 있지 않다”며 “새롭게 형성되는 가치사슬을 우리가 주도할 수 있는 기회를 잡아야 한다”고 밝히고 있다.', '최근 일본 정부와 재계 언론은 하나같이 ‘디지털’을 외치고 있다. 디지털 지체로 생긴 한국과의 부가가치 열위를 만회하겠다고 읽힌다. 새로운 가치사슬과 부가가치를 향해 애쓰는 노력이 엿보인다. 일본의 수출규제로 촉발된 부품 소재 국산화도 나름 의미 있고 필요한 일이다. 하지만 일본보다 부가가치를 높일 수 있는 5G 등 지식집약적인 4차 산업에 매진하는 것이 더 필요할지 모른다.', 'ohchoon@hankyung.com\n\t\t\t\t\t\t\t\t']"
0,1,2.100601e+06,20190820,"['\n', '\n', '\n', '\n', '\n', '\n', '\n ..."
1,2,2.100601e+06,20190820,"['\n', '\n', '\n', '\n', '\n', '\n', '\n ..."
2,3,2.100601e+06,20190820,"['\n', '\n', '\n', '\n', '\n', '\n', '\n ..."
3,4,2.100601e+06,20190820,['\n\t\t\t\t\t\t\t\t정부가 어제 국무회의를 열고 ‘사회적 기업 육성...
4,5,2.100601e+06,20190820,['\n\t\t\t\t\t\t\t\t정부가 조만간 확정할 예정인 ‘2020년 예산안...
...,...,...,...,...
1115,1116,2.100601e+06,20190819,

In [46]:
pd.read_csv('NewsResult_20190711-20190720 (1).csv')

,0,뉴스 식별자,일자,단어,본문
0,1,2.100601e+06,20190720,"교급식,왕,사진,방송,면,오늘,일,토,방송,고교,급,식,왕,결승전,눈길,오늘,방송,...","['\n', '\n', '\n', '\n', '\n', '\n', '고교급식왕(사진..."
1,2,2.100601e+06,20190720,"강호동,사진,방송,강호동,팬,앞,눈물,일,방송,강식당,강호동,팬,진심,감동,눈물,모...","['\n', '\n', '\n', '\n', '\n', '강호동/사진=tvN', '..."
2,3,2.100601e+06,20190720,"지성,세,영,사진,의사,요한,막강,의사,케미,가동,금,토,드라마,의사,요한,회,시청...","['\n', '\n', '\n', '\n', '\n', '\n', '지성, 지세영(..."
3,4,2.100601e+06,20190720,"에이스,바론,사진,엔터테인먼트,제공,그룹,멤버,에,이,스,바론,컴백,특별,시,구,시...","['\n', '\n', '\n', '\n', '\n', '\n', '에이스, 바론(..."
4,5,2.100601e+06,20190720,"양준혁,사진,프로,야구,선수,출신,양준혁,박찬호,언급,눈길,일,오후,방송,예능,프로...","['\n', '\n', '\n', '\n', '\n', '양준혁/사진=JTBC', ..."
...,...,...,...,...,...
1095,1096,2.100601e+06,20190718,"차트,분석,단기,중기,평선,정배열,상승세,시간,차트,흐름,단기,중기,평선,정배열,상...","['\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n..."
1096,1097,2.100601e+06,20190718,"가수,하성,운,오후,서울,상암동,센터,엠카운트,드라이,카메라,리허설,참석,포토타임,...","['\n', '\n', '\n', '\n', '\n', '\n', ""\n ..."
1097,1098,2.100601e+06,20190718,"녹원씨,엔아,이,세계,전자,회사,세계,최초,평면,연성,인쇄,회로,기판,회로,보호,잉...","['\n', '녹원씨엔아이', '(6,010 -1.15%)', '는 세계적 전자회사..."
1098,1099,2.100601e+06,20190718,"일본,정부,강제,징용,문제,해법,제안,제국,중재,위원회,구성,시한,일,정부,거부,입...","['\n', '\n', '\n', '\n', '\n', '일본 정부가 강제징용 문제..."
